In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [40]:
import os

# Set environment variables
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_PROJECT'] = 'cortex'

# Get keys from the environment
langchain_api_key = os.getenv("LANGCHAIN_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")

if langchain_api_key:
    os.environ['LANGCHAIN_API_KEY'] = langchain_api_key
else:
    raise ValueError("LANGCHAIN_API_KEY is not set in the environment.")

if groq_api_key:
    os.environ['GROQ_API_KEY'] = groq_api_key
else:
    raise ValueError("GROQ_API_KEY is not set in the environment.")

Retriever

In [20]:
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from sentence_transformers import SentenceTransformer

# Custom embedding class to wrap the SentenceTransformer model
class SentenceTransformerEmbedding:
    def __init__(self, model_name, device='cpu'):
        self.model = SentenceTransformer(model_name, device=device)
    
    def embed_documents(self, texts):
        return [self.model.encode(text) for text in texts]
    
    def embed_query(self, query):
        # Ensure this returns a 2D array (even if only one query)
        return [self.model.encode(query)]

# List of URLs to load documents from
urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

# Load documents from the URLs
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

# Split documents into smaller chunks for easier processing
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=250, chunk_overlap=0)
doc_splits = text_splitter.split_documents(docs_list)

# Initialize custom embedding class
model_name = "BAAI/bge-small-en"
embedding_function = SentenceTransformerEmbedding(model_name=model_name, device="cpu")

# Store the split documents in Chroma vector database using the custom embedding function
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding=embedding_function  # Pass the instance of SentenceTransformerEmbedding
)

# Create a retriever for querying the vectorstore
retriever = vectorstore.as_retriever()

# Optional: You can print the retriever to check if it's working correctly
print(retriever)

tags=['Chroma', 'SentenceTransformerEmbedding'] vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x17acbbe80> search_kwargs={}


LLMs

In [30]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from pydantic import BaseModel, Field
import numpy as np

# Data model
class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""
    binary_score: str = Field(..., description="Documents are relevant to the question, 'yes' or 'no'")

# LLM with function call 
llm = ChatGroq(temperature=0)
structured_llm_grader = llm.with_structured_output(GradeDocuments)

# Prompt 
system = """You are a grader assessing relevance of a retrieved document to a user question. \n 
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""

grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)

retrieval_grader = grade_prompt | structured_llm_grader


In [41]:
question = "agent memory"
docs = retriever.get_relevant_documents(question)
doc_txt = docs[1].page_content
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

ValueError: Expected embeddings to be a list of floats or ints, a list of lists, a numpy array, or a list of numpy arrays, got [[array([-5.06070517e-02,  5.97463269e-03,  2.03445759e-02, -2.69595329e-02,
       -2.03757975e-02,  2.80002970e-02,  5.12210988e-02,  2.03084257e-02,
        5.20406887e-02, -4.87228297e-03,  7.47223245e-03, -1.37139037e-02,
        7.51490444e-02,  5.91166541e-02,  1.55815640e-02,  1.40016228e-02,
       -3.54385525e-02,  5.04436828e-02,  4.38578706e-03, -3.48372012e-02,
        2.74588075e-02, -6.84176162e-02, -8.07437953e-03, -1.36269713e-02,
       -6.49731532e-02,  2.87238527e-02, -4.87621687e-02, -2.01405920e-02,
       -3.68303619e-02, -1.67949498e-01,  1.84508990e-02,  1.47046603e-03,
        2.28697825e-02,  1.14864006e-03,  9.20567941e-03, -2.64571495e-02,
       -2.37961505e-02,  1.43749304e-02, -4.25855666e-02,  8.24343506e-03,
        4.37707528e-02,  2.43634116e-02, -3.13222818e-02, -2.15302575e-02,
       -1.84855163e-02, -4.35628854e-02,  5.97